In [1]:
#
# Revisar manualmente as imagens em C:\mestrado\datasets\samples_font  ok
#
#
# Pegar todas as imagens em final-32K-8lines do tipo sync_**.jpg e sobrescrever
# usando como texto a sequencia no id correspondente em portugues.
#
# Para os fontes carregar a lista de fontes cujas imagens existem em samples_font
# buscando as fontes em C:/fonts, mais as fontes do windows usado anteriormente
# Usar as fontes aleatoriamente...
#


In [ ]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
import random
import pickle
import cv2
import os.path
from os import path
import shutil
from glob import glob
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.display import clear_output, display
from collections import Counter
import builtins

from sklearn.utils import shuffle

#
# gera imagem com jogadas escritas com fonte manuscrito
#
# -*- coding: utf-8 -*-
import PIL
from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

# print(tf.__version__) 

In [3]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [4]:
# https://gist.github.com/erniejunior/601cdf56d2b424757de5
def elastic_transform_2(image, alpha, sigma,random_state=None):
#     print( alpha, sigma)
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
#     print (x.shape)
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    return distored_image.reshape(image.shape)

In [5]:
def transform( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))

In [ ]:
# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split()[0:32] for ln in lines]

# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
# with open(pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split()[0:32] for ln in en_lines]

# pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split() for ln in lines]

# en_pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_eng.txt"
# with open(en_pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split() for ln in en_lines]

In [ ]:
# print( lines[0], en_lines[0])

In [ ]:
# print( len( lines), len(en_lines))

In [ ]:
#
# le todos os fontes
#

In [6]:
new_fonts_files= glob( 'C:/mestrado/datasets/samples_font--2/*.ttf.jpg')

In [7]:
print( len( new_fonts_files))

120


In [8]:
new_fonts_path= [
    os.path.join( 'C:/mestrado/datasets/fonts', 
                 Path( f).name.replace( '.jpg', '')) for f in new_fonts_files]

In [9]:
new_fonts= [ImageFont.truetype(f, 30) for f in new_fonts_path]

In [10]:
fonts= [
    ImageFont.truetype('c:/Windows/Fonts/BRADHITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/ITCKRIST.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/LHANDW.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/Inkfree.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/OCRAEXT.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/TEMPSITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/PAPYRUS.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/MAIAN.TTF', 30)
] 

In [11]:
all_fonts= new_fonts + fonts

In [12]:
print( len( new_fonts), len( fonts), len( all_fonts))

120 8 128


In [13]:
def showbig( img, name= 'noname'):
    plt.figure(figsize = (10,10))    
    plt.imshow( img)
    plt.title( name)
    plt.show()

In [15]:
def sp_noise(npimage,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(npimage.shape,np.uint8)
    thres = 1 - prob 
    for i in range(npimage.shape[0]):
        for j in range(npimage.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = random.randint( 0, 128)
            else:
                output[i][j] = npimage[i][j]
    return output

def sp_noise_pil( pilimg, prob):
#     print( 'noise', prob)
    img= pilimg
    img= np.asarray( img)
    img= sp_noise(img, prob)
    img= PIL.Image.fromarray( img)
    return img

def distorce( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
#     print( 'distorce')
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
    

# Rotate the foreground
def rotate( pilimg):
#     print( 'rotate')
    foreground= pilimg
    angle_degrees = random.uniform(-2.5,2.5)
#     print( angle_degrees)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True, fillcolor = (255,255,255))

    # Scale the foreground
#     scale = random.random() * .5 + .5 # Pick something between .5 and 1
#     scale = 1
#     new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
#     foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    return foreground
#     show( foreground)
    

def random_transform( pilimg, p):
    img= pilimg
    if( random.random()< 0.8):
        img= rotate( img)
#     if( random.random()< 0.1):
#         img= distorce( img)
    if( random.random()< 0.25):
        img= sp_noise_pil( img, random.uniform( 0.001, 0.0001))
    
    return img

In [ ]:
# pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v5/*/labels/*.pgn')
# pgn_files= [f for f in pgn_files if 'scan' in f]
# print( len( pgn_files))

# all_pgn= []
# for f in pgn_files:
#     all_pgn.append( read_label( f))
	
# all_pgn= [p.split()[0:16] for p in all_pgn]
# flat_all_pgn = [t for _list in all_pgn for t in _list]

# count16= Counter( flat_all_pgn)

# print( 'len( count16)', len( _count16))  #esperado 462
# counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
# print( counted16)



In [34]:
pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v7/test/labels/*.pgn')

all_pgn= []
for f in pgn_files:
    all_pgn.append( read_label( f))
	
lines_test= [p.split()[0:16] for p in all_pgn]

In [35]:
lines_test

[['e4',
  'c5',
  'Nf3',
  'e6',
  'd4',
  'cxd4',
  'Nxd4',
  'a6',
  'c4',
  'Qc7',
  'Nc3',
  'Nf6',
  'Be3',
  'Bb4',
  'Qd2',
  'Nxe4'],
 ['e4',
  'c5',
  'Nf3',
  'd6',
  'Bb5+',
  'Bd7',
  'Bxd7+',
  'Nxd7',
  'O-O',
  'Ngf6',
  'Re1',
  'e5',
  'c3',
  'Be7',
  'd3',
  'O-O'],
 ['Nf3',
  'Nf6',
  'g3',
  'd5',
  'c4',
  'e6',
  'b3',
  'Be7',
  'Bb2',
  'O-O',
  'Bg2',
  'Nbd7',
  'O-O',
  'c6',
  'd3',
  'Re8'],
 ['c4',
  'c5',
  'Nc3',
  'Nc6',
  'g3',
  'g6',
  'Bg2',
  'Bg7',
  'Nf3',
  'd6',
  'O-O',
  'e5',
  'd3',
  'Nge7',
  'Rb1',
  'O-O'],
 ['d4',
  'd5',
  'Bf4',
  'Nf6',
  'e3',
  'c5',
  'c3',
  'Qb6',
  'Qb3',
  'c4',
  'Qc2',
  'g6',
  'Nd2',
  'Nc6',
  'e4',
  'e6'],
 ['e4',
  'e5',
  'Nf3',
  'd6',
  'Nc3',
  'Nc6',
  'Bc4',
  'Be7',
  'h3',
  'Nf6',
  'd4',
  'exd4',
  'Nxd4',
  'Nxd4',
  'Qxd4',
  'O-O'],
 ['e4',
  'e5',
  'Nf3',
  'Nc6',
  'Bc4',
  'Nf6',
  'Ng5',
  'd5',
  'exd5',
  'Na5',
  'Bb5+',
  'c6',
  'dxc6',
  'bxc6',
  'Bd3',
  'Nd5'],
 ['e4',
  '

In [36]:
flat_test_pgn = [t for _list in lines_test for t in _list]

In [40]:
[p for p in flat_test_pgn if p not in words]

[]

In [17]:
# le os primeiros 16
pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_eng.txt"
with open(pgnfile) as f:
	lines = [line.rstrip() for line in f]
lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
lines= [ln[8:] for ln in lines]
lines16= [ln.split()[:16] for ln in lines]
flat_all_pgn = [t for _list in lines16 for t in _list]
count16= Counter( flat_all_pgn)
words= [item for item in count16]


count16 Counter({'O-O': 25033, 'Nf6': 24489, 'Nf3': 24462, 'd4': 23906, 'e4': 19723, 'Nc3': 19474, 'd5': 15863, 'e6': 14668, 'Nc6': 14029, 'c5': 13768, 'c4': 13249, 'd6': 12986, 'e5': 11864, 'g6': 9536, 'Bg7': 8945, 'Be7': 7379, 'cxd4': 7311, 'c6': 6719, 'g3': 6466, 'Bg2': 6088, 'a6': 5914, 'e3': 5802, 'Nxd4': 5321, 'Bd3': 5155, 'c3': 4774, 'Bg5': 4571, 'Be2': 4566, 'exd5': 4202, 'Nbd7': 4052, 'd3': 3980, 'Be3': 3889, 'cxd5': 3577, 'Bc4': 3185, 'b6': 3055, 'b5': 2694, 'f4': 2540, 'Bg4': 2486, 'Bb4': 2484, 'h6': 2453, 'Bb7': 2449, 'h3': 2371, 'Bb5': 2360, 'a3': 2228, 'Nbd2': 2073, 'b3': 1878, 'Bf5': 1846, 'Bd6': 1832, 'Bf4': 1791, 'Bd7': 1789, 'Qc7': 1739, 'Nd7': 1671, 'Nxe4': 1664, 'f5': 1633, 'Qc2': 1621, 'dxc4': 1619, 'exd4': 1609, 'Qd2': 1584, 'Nxd5': 1574, 'Re1': 1485, 'dxe5': 1480, 'f3': 1442, 'Nd2': 1412, 'Bc5': 1404, 'a4': 1321, 'Bb2': 1300, 'dxe4': 1300, 'Qb6': 1293, 'Bb3': 1261, 'Ne7': 1260, 'Bd2': 1229, 'Nge2': 1187, 'Nd5': 1149, 'Nge7': 1116, 'Ne4': 1106, 'Qe2': 1084, 'Be6':

In [18]:
print( 'count16', count16)

count16 Counter({'O-O': 25033, 'Nf6': 24489, 'Nf3': 24462, 'd4': 23906, 'e4': 19723, 'Nc3': 19474, 'd5': 15863, 'e6': 14668, 'Nc6': 14029, 'c5': 13768, 'c4': 13249, 'd6': 12986, 'e5': 11864, 'g6': 9536, 'Bg7': 8945, 'Be7': 7379, 'cxd4': 7311, 'c6': 6719, 'g3': 6466, 'Bg2': 6088, 'a6': 5914, 'e3': 5802, 'Nxd4': 5321, 'Bd3': 5155, 'c3': 4774, 'Bg5': 4571, 'Be2': 4566, 'exd5': 4202, 'Nbd7': 4052, 'd3': 3980, 'Be3': 3889, 'cxd5': 3577, 'Bc4': 3185, 'b6': 3055, 'b5': 2694, 'f4': 2540, 'Bg4': 2486, 'Bb4': 2484, 'h6': 2453, 'Bb7': 2449, 'h3': 2371, 'Bb5': 2360, 'a3': 2228, 'Nbd2': 2073, 'b3': 1878, 'Bf5': 1846, 'Bd6': 1832, 'Bf4': 1791, 'Bd7': 1789, 'Qc7': 1739, 'Nd7': 1671, 'Nxe4': 1664, 'f5': 1633, 'Qc2': 1621, 'dxc4': 1619, 'exd4': 1609, 'Qd2': 1584, 'Nxd5': 1574, 'Re1': 1485, 'dxe5': 1480, 'f3': 1442, 'Nd2': 1412, 'Bc5': 1404, 'a4': 1321, 'Bb2': 1300, 'dxe4': 1300, 'Qb6': 1293, 'Bb3': 1261, 'Ne7': 1260, 'Bd2': 1229, 'Nge2': 1187, 'Nd5': 1149, 'Nge7': 1116, 'Ne4': 1106, 'Qe2': 1084, 'Be6':

In [19]:
print( len( words), words)

776 ['d4', 'f5', 'Nf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Nf6', 'b4', 'c6', 'c4', 'd5', 'Qb3', 'Be6', 'c5', 'Ne4', 'e4', 'd6', 'cxd4', 'Nxd4', 'Nc3', 'a6', 'Bc4', 'e6', 'Bb3', 'b5', 'O-O', 'Be7', 'e3', 'Nbd2', 'dxc5', 'Bxc5', 'Qc2', 'Ba7', 'a3', 'Nc6', 'Be3', 'Be2', 'Nb8', 'a4', 'e5', 'Bf5', 'c3', 'Nge7', 'Qb6', 'Bb5', 'Nxe4', 'Nd6', 'Bxc6', 'dxc6', 'dxe5', 'Nf5', 'h6', 'b3', 'b6', 'Bb2', 'Bb7', 'cxd5', 'Nxd5', 'Ndb4', 'Nxc6', 'Qxd1+', 'Nb5', 'Bg5', 'Nfd7', 'Bxe7', 'Qxe7', 'Bd3', 'exd4', 'h3', 'a5', 'f4', 'Bc5', 'Nxe5', 'dxe4', 'Bxf7+', 'Kf8', 'Ng4', 'Nxg4', 'Qh4+', 'Qe2', 'Qe7', 'd3', 'Nbd7', 'Qxe2+', 'Ng5', 'exd5', 'Na5', 'Bb5+', 'bxc6', 'Qf3', 'Bb4+', 'Bd2', 'Bxc3', 'Rc1', 'h5', 'Nh6', 'Bg4', 'Ba4', 'Nc5', 'Qxd4', 'Nh3', 'Nf4', 'cxb5', 'bxa6', 'Qxd5', 'Qd4', 'f6', 'Bf4', 'Qc7', 'fxe4', 'Bd6', 'Ne5', 'Nd2', 'Bd7', 'Nd5', 'Qe4', 'Bxd3', 'Qxd3', 'Ne2', 'f3', 'Nge2', 'Qd2', 'Re8', 'Ne7', 'axb5', 'Ngf3', 'h4', 'Qd7', 'Rb1', 'Nb3', 'Bb4', 'Bxc3+', 'bxc3', 'Qe8', 'Bh4', 'Ndb5', 'Nxc3', 'Nce2', 'Q

In [20]:
# 
# transforma de portugues para ingles, para gerar pgn
def pt_to_pgn( w):
    # R=> K
    # T=> R
    # C=> N
    # B=> B
    # D=> Q

#     print( w, '=> ')
    png= w.replace( 'R', 'K').replace( 'T', 'R').replace( 'C', 'N').replace( 'D', 'Q')
#     print( png)
    return png

def to_pgn( label):
    return [pt_to_pgn( w) for w in label]

In [21]:
def pgn_to_pt( w):
    # R => T
    # K => R
    # N => C
    # Q => D
    
    return w.replace( 'R', 'T').replace( 'K', 'R').replace( 'N', 'C').replace( 'Q', 'D')

In [22]:
def random_label_from_vocab():
    label= []
    for i in range( 0, 16):
        label.append( 
            pgn_to_pt( words[random.randrange( 0, len( words))])
        )
    return label


print( random_label_from_vocab())

['exf3', 'Dc5', 'Tf1', 'f3', 'Txc1', 'Cbxd2', 'Bh4+', 'Re2', 'Cce2', 'Bf3', 'f4', 'Bc5', 'Da4', 'Cfe7', 'C5b6', 'Dxd2+']


In [23]:
dd= {w:0 for w in words}

In [24]:
for i in range( 0, 20000*16):
    w= words[random.randrange( 0, len( words))]
    dd[ w]= dd[ w]+1
    

In [25]:
print( dd)

{'d4': 441, 'f5': 444, 'Nf3': 490, 'g6': 426, 'g3': 402, 'Bg7': 422, 'Bg2': 464, 'Nf6': 420, 'b4': 368, 'c6': 413, 'c4': 427, 'd5': 412, 'Qb3': 386, 'Be6': 411, 'c5': 351, 'Ne4': 405, 'e4': 404, 'd6': 440, 'cxd4': 377, 'Nxd4': 406, 'Nc3': 433, 'a6': 405, 'Bc4': 384, 'e6': 394, 'Bb3': 415, 'b5': 391, 'O-O': 411, 'Be7': 450, 'e3': 447, 'Nbd2': 391, 'dxc5': 397, 'Bxc5': 385, 'Qc2': 433, 'Ba7': 396, 'a3': 413, 'Nc6': 382, 'Be3': 406, 'Be2': 434, 'Nb8': 448, 'a4': 416, 'e5': 421, 'Bf5': 407, 'c3': 399, 'Nge7': 419, 'Qb6': 404, 'Bb5': 375, 'Nxe4': 439, 'Nd6': 402, 'Bxc6': 401, 'dxc6': 413, 'dxe5': 397, 'Nf5': 383, 'h6': 389, 'b3': 407, 'b6': 465, 'Bb2': 412, 'Bb7': 384, 'cxd5': 407, 'Nxd5': 440, 'Ndb4': 408, 'Nxc6': 457, 'Qxd1+': 387, 'Nb5': 397, 'Bg5': 410, 'Nfd7': 409, 'Bxe7': 403, 'Qxe7': 396, 'Bd3': 400, 'exd4': 434, 'h3': 401, 'a5': 417, 'f4': 416, 'Bc5': 403, 'Nxe5': 415, 'dxe4': 390, 'Bxf7+': 426, 'Kf8': 411, 'Ng4': 422, 'Nxg4': 409, 'Qh4+': 400, 'Qe2': 417, 'Qe7': 427, 'd3': 411, 'Nb

In [26]:
min( [v for (k,v) in dd.items()])

344

In [27]:
max( [v for (k,v) in dd.items()])

490

In [30]:
len( all_fonts)

128

In [32]:
img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
dctx = ImageDraw.Draw(img)  # create drawing context
dh_top= 7
dh_bottom= 13
h= (img.height - dh_top - dh_bottom)/25.0
w= img.width/4.0

margin_left= w/4
margin_top= h/6

# def resize_fixed( img):
#     _dd = (271, 230) #width, height
#     return cv2.resize(img, _dd, interpolation = cv2.INTER_AREA) 

def generate_for( line):
#     if fnt is None:
#         fnt= random.choice( fonts)
#     print( fnt.getname())
    
    img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
    dctx = ImageDraw.Draw(img)  # create drawing context

    i= 0
    for ln in range( 0, 8):
        for col in range( 0, 2):
            if i>= len( line):
                del dctx
                return img
        
            fnt= random.choice( all_fonts)
            txtsz = dctx.textsize(line[i], fnt)
        
            x= w* col + (w - txtsz[0])/2
            if random.random()< 0.7:
                xmargem= int( (w - txtsz[0])/3)
                x+= random.randint( -xmargem, xmargem)
            y= dh_top + h*ln + (h - txtsz[1])/2 #+ margin_top

            dctx.text( (x, y), line[i], font=fnt, fill="#000000")
            i= i+1
            
    del dctx 
    return img

def cut( img):
    h= 300
    w= 345

    img1= img[:h, :w]
    return img1

def generate_one( label): #pt_line, pgn, no, folder):
#     id_font= no%len( all_fonts)
    img= generate_for( label)
    img= cut( np.asarray(img))
    img= PIL.Image.fromarray(np.uint8(img))
    return img


# def __generate():
#     _files= label_files
# #     print( 'Gerando ', len( _files))
#     for i in range( 0, len( _files)):
#         if i%100== 0:
#             print( i)
#         img_file= _files[i].replace( 'labels', 'images.new').replace( '.pgn', '.jpg') 
#         img= generate_one( i, labelof( _files[i])[0:16], img_file)
        
#         img= random_transform( img, 0.3)
#         img.save( img_file)
# #         print( img_file)
# #         show( img__)
# #         show( transform( img__))
# #         return img
    
# __generate()
# img1= __generate()
# show( img1)
# pilimg= img1
# npimg= np.asarray( img1)


def __generate_random( N):
    dataset_dir= 'C:/mestrado/datasets/unified/random-32k-8lines--v7'
    
    N_train= int(N*0.8)
    print( 'N', N, 'N_train', N_train)
    
    for i in range( 0, N):
        if( i%100== 0):
            print( i)
        label_pt= random_label_from_vocab()
        img= generate_one( label_pt)
        img= random_transform( img, 0.3)
        pgn= to_pgn( label_pt)

        folder= 'train' if i< N_train else 'valid'
        
        # save
#         show( img)
        img_file= os.path.join( dataset_dir, folder, 'images', 'random-syn-'+ str(i+1).zfill( 5)+'.jpg')
        img.save( img_file)
        write_label( img_file.replace( 'images', 'labels').replace( '.jpg', '.pgn'), 
                    " ".join( to_pgn( label_pt)))

__generate_random( 30000)

N 30000 N_train 24000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
1810

In [33]:
def gererate_all_labels():
    pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v7/*/labels/*.pgn')
    all_pgn= []
    for f in pgn_files:
        all_pgn.append( read_label( f))
        
    all_pgn= [' '.join( p.split()[0:16]) for p in all_pgn]
    
    write_label( 'C:/mestrado/datasets/unified/random-32k-8lines--v7/all-labels-.pgn', 
           '\n'.join( all_pgn))
    
gererate_all_labels()

In [ ]:
pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v6/*/labels/*.pgn')

In [ ]:
all_pgn= []
for f in pgn_files:
    all_pgn.append( read_label( f))

pat= "Nc3 d5 c4"

In [ ]:
for i in range( 0, len( all_pgn)):
    if all_pgn[i].startswith( "Nc3 d5"):
        print( i, all_pgn[i])